In [1]:
# Import required modules
import pandas as pd
import numpy as np
from ds_dev_tools.awstools import FileDoggo, S3Location
import plotly.express as px
import time
import boto3

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# Define date range for queries
start_datetime = '2024-12-07 00:00:00'
end_datetime = '2025-01-07 23:59:59'
start_year = 2024
end_year = 2025
start_month = 12
end_month = 1
start_day = 7
end_day = 7

In [3]:
# Initialize Athena client
athena_client = boto3.client('athena')

output_location = 's3://production-datalake-stac-datalakeskipprbucket4a91-jqc93kmxmqsm/output/dev/'

# Method to execute athena query
def query_athena(query_string):
    response = athena_client.start_query_execution(
        QueryString=query_string,
        QueryExecutionContext={
            'Database': 'datalake'
        },
        ResultConfiguration={
            'OutputLocation': output_location
        }
    )

    # Get the QueryExecutionId to check the query status and get results
    query_execution_id = response['QueryExecutionId']
    query_status = 'QUEUED'

    while query_status in ['RUNNING', 'QUEUED']:
        query_status_response = athena_client.get_query_execution(
            QueryExecutionId=query_execution_id
        )
        query_status = query_status_response['QueryExecution']['Status']['State']

        if query_status == 'FAILED':
            raise Exception("Athena query failed.")

        # Add a delay so we don't overwhelm the service
        if query_status in ['RUNNING', 'QUEUED']:
            time.sleep(5)


    with FileDoggo(S3Location(output_location).join(f'{query_execution_id}.csv')) as f:
        df = pd.read_csv(f)

    return df

In [5]:
device_data_query = f'''
    SELECT
        imei,
        truck_registration,
        system_datetime_posix_utc_seconds,
        system_cpu_temperature_degc,
        drum_data_valid,
        drum_vector_rpm AS drum_speed_mean_rpm,
        system_operating_mode,
        pressure_a_bar_data_valid,
        pressure_a_bar_mean,
        pressure_b_bar_data_valid,
        pressure_b_bar_mean,
        temperature_module_data_valid,
        temperature_module_surface_temperature_degc,
        temperature_module_speed_mean_rpm,
        truck_gearbox_ratio AS calibration_gearbox_ratio,
        truck_id,
        truck_motor_displacement_cm3 AS calibration_motor_displacement_cm3,
        truck_motor_efficiency AS calibration_motor_efficiency,
        truck_rmc_provider,
        water_flowmeter_data_valid AS water_valid,
        water_flowmeter_total_volume_m3 AS water_cumu_vol,
        water_flowmeter_flow_rate_m3_hr
    FROM "datalake"."device_data"
    WHERE year BETWEEN {start_year} AND {end_year}
        AND month IN ({start_month}, {end_month})
        AND system_datetime_posix_utc_seconds > CAST('{start_datetime}' AS timestamp)
        AND system_datetime_posix_utc_seconds < CAST('{end_datetime}' AS timestamp)
        AND truck_registration IN ('BJ17WUY', 'RK22FNM', 'KU68BYL', 'RX16WXR', 'KX67RKO')
    ORDER BY system_datetime_posix_utc_seconds ASC
'''

device_df = query_athena(device_data_query)

In [6]:
# Convert to datetime
device_df['datetime_utc'] = pd.to_datetime(device_df.system_datetime_posix_utc_seconds).dt.tz_localize('utc')

device_df = device_df[
    (device_df.datetime_utc >= start_datetime) &
    (device_df.datetime_utc <= end_datetime)
]

In [ ]:
device_df = device_df.sort_values(by=['truck_registration', 'datetime_utc'])
trucks = device_df.truck_registration.unique()

for truck in trucks:
    device_df.loc[device_df.truck_registration == truck, 'prev_system_operating_mode'] = device_df.loc[device_df.truck_registration == truck, 'system_operating_mode'].shift(1)
    device_df.loc[device_df.truck_registration == truck, 'prev_system_datetime_posix_utc_seconds'] = device_df.loc[device_df.truck_registration == truck, 'system_datetime_posix_utc_seconds'].shift(1)
    device_df.loc[device_df.truck_registration == truck, 'prev_system_operating_mode'] = device_df.loc[device_df.truck_registration == truck, 'prev_system_operating_mode'].fillna('N/A')
    device_df.loc[device_df.truck_registration == truck, 'prev_datetime_utc'] = device_df.loc[device_df.truck_registration == truck, 'datetime_utc'].fillna(pd.to_datetime('2024-12-07 00:00:00').tz_localize('utc'))

device_df['was_idle_flag'] = device_df.apply(
    lambda x: 1 if (
        (x.system_operating_mode in ['Measuring', 'Condensation Control'])
        & ('Periodic Connect' in x.prev_system_operating_mode)
    )  else (
        1 if (
            (x.system_operating_mode in ['Measuring', 'Condensation Control'])
            & (x.prev_system_operating_mode == 'Measuring Mode Engine Off')
            & ((x.datetime_utc - x.prev_datetime_utc).seconds > 600)
        ) else 0
    ), axis=1
)

In [22]:
device_df[['truck_registration', 'was_idle_flag']].groupby('truck_registration').sum()

,was_idle_flag
truck_registration,
BJ17WUY,16
KU68BYL,15
KX67RKO,15
RK22FNM,14
RX16WXR,16
